In [26]:
import pandas as pd
import re
import MeCab
from elasticsearch import Elasticsearch

In [41]:
neologd_dict_dir="C:/Tools/MeCab/dic/mecab-ipadic-neologd/"
csv_dir="C:/Users/00991442/Desktop/recommend_letter_utf8.csv"
es_host = "localhost:9200"
index_name="docs"


wakati = MeCab.Tagger(f"-Owakati -d{neologd_dict_dir}")
conn = Elasticsearch([es_host])

In [30]:
df = pd.read_csv(csv_dir, encoding="utf8")
#df[0:1]

In [24]:
letters = df[["会員番号", "レコメンドレター", "ふりがな(名)"]].loc[df['レコメンドレター'].notna()].rename(columns={"レコメンドレター":"text", "ふりがな(名)": "name", "会員番号": "member_no"})
data = [*letters.T.to_dict().values()]
# data[0:10]
#letters[0:10].to_json("letters.json", force_ascii=False, orient="records")

In [25]:
re_white_space = re.compile("\s+")

def to_wakati(text):
    text = wakati.parse(text)
    return re_white_space.split(text)

In [23]:
def replace_name(name, text):
    re_name_expression = re.compile(f"({name}(さん|様|さま))")
    return re_name_expression.sub("NAME", text)

def to_index_data(record):
    doc_id = record["member_no"]
    name = record["name"]
    text = record["text"]
    replaced = replace_name(name, text)
    wakati = to_wakati(replaced)
    return { "doc_id": doc_id, "name": name, "text": replaced, "wakati": wakati  }



# [*map(lambda x: to_index_data(x), data[0:1] )]

In [44]:
limit = -1
run = False
if run:
    for record in data[0:limit]:
        doc = to_index_data(record)
        # print(f"putting {doc['doc_id']}")
        res = conn.index(index=index_name, id=doc["doc_id"], body=doc)
